# Building the pipeline for Advanced Lane Line finding

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
%matplotlib inline

## Camera Calibration

In [2]:
# prepare object points for a 9x6 chess board with z=0, like (0,0,0), (1,0,0), ...
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

# create empty object points and image points arrays
objpoints = []
imgpoints = [] 

# Make a list of calibration images
files = os.listdir("camera_cal/")

# Step through the image list and search for corners
for file in files:
    if file[0:11] == "calibration": #execute only on files with the prefix "calibration"
        img = mpimg.imread("camera_cal/"+file)
        img_size = (img.shape[1], img.shape[0])
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # convert to grayscale from RGB as were read with mpimg

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9, 6), None)
    
        # If found, draw corners, compute calibration and save images to folder
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
        
# Compute the camera calibration and distortion coefficients
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)

# Undistort a test image with the parameters from cv2.calibrateCamera function
img = mpimg.imread("camera_cal/calibration2.jpg")
img = cv2.undistort(img, mtx, dist, None, mtx)
r,g,b = cv2.split(img)    #since image gets recombined as BGR images, use cv2.split to put into RGB format
img = cv2.merge((b,g,r))
cv2.imwrite("camera_cal/undistort_calibration2.jpg", img);    
    

## First build a pipeline and test it on still test images

## Apply distortion correction to each test image

In [3]:
def undistorted(img):
    # Use calibration parameters calculated before in cv2.undistort()
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    r,g,b = cv2.split(undist)  #since image gets recombined as BGR images, use cv2.split to put into RGB format
    undist = cv2.merge((b,g,r))
    return undist

# Make a list of the distorted test images
files = os.listdir("test_images/")

# Step through the image list and write back each undistorted test image
for file in files:
    if file[0:6] != "output_": #execute only on files that were not already created and have the prefix "output"
        img = mpimg.imread("test_images/"+file)
        img = undistorted(img)
        cv2.imwrite("output_images/output_undistorted_"+file,img)    #write output image


## Create binary images using color transform and gradient methods

In [4]:
def binary_img(img, s_thresh=(254, 255), b_thresh=(254, 255), sx_thresh=(20, 80)):
    # Use contrast correction to fight excessive darkness or brightness
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img[:,:,0] = cv2.equalizeHist(img[:,:,0])
    img = cv2.cvtColor(img, cv2.COLOR_YUV2RGB) 
    
    # Convert to HLS color space and separate the S channels
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    # Convert to LAB color space and separate the B channel 
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    b_channel = lab[:,:,2]
    
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold S-color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # Threshold B-color channel
    b_binary = np.zeros_like(b_channel)
    b_binary[(b_channel >= b_thresh[0]) & (b_channel <= b_thresh[1])] = 1
    
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (b_binary == 1) | (sxbinary == 1)] = 1
    
    return combined_binary

In [5]:
# Make a list of the distorted test images
files = os.listdir("test_images/")

# Step through the image list and write back each test image as an undistorted and binary image
for file in files:
    if file[0:6] != "output_": #execute only on files that were not already created and have the prefix "output"
        img = mpimg.imread("test_images/"+file)
        undistort = undistorted(img)
        binary = binary_img(undistort)
        color_binary = np.dstack((binary, binary, binary)) * 255 # stack each channel for RGB output
        kernel = np.ones((2,2),np.uint8)
        color_binary = cv2.morphologyEx(color_binary, cv2.MORPH_OPEN, kernel)
        # plt.imshow(color_binary)
        cv2.imwrite("output_images/output_binary_"+file,color_binary)    #write output image

## Apply a perspective transform to the image

In [6]:
# Use hardcoded course and destination points
src = np.float32(
    [[img_size[0] / 2 - 55, img_size[1] / 2 + 100],
    [img_size[0] / 6 - 10, img_size[1]],
    [img_size[0] * 5 / 6 + 60, img_size[1]],
    [img_size[0] / 2 + 55, img_size[1] / 2 + 100]])
dst = np.float32(
    [[img_size[0] / 4, 0],
    [img_size[0] / 4, img_size[1]],
    [img_size[0] * 3 / 4, img_size[1]],
    [img_size[0] * 3 / 4, 0]])

# Given src and dst points, calculate the perspective transform matrices
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)

# Define a function that takes an image, number of x and y points, 
# camera matrix and distortion coefficients
def warp(img, mtx, dist):
    # Use the OpenCV undistort() function to remove distortion
    undist = cv2.undistort(img, mtx, dist, None, mtx)
        
    img_size = (img.shape[1], img.shape[0])
        
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(undist, M, img_size)
    
    # Return the resulting image and matrix
    return warped

In [7]:
# Make a list of the distorted test images
files = os.listdir("test_images/")

# Step through the image list and write back each test image as an undistorted and binary image
for file in files:
    if file[0:6] != "output_": #execute only on files that were not already created and have the prefix "output"
        img = mpimg.imread("test_images/"+file)
        undistort = undistorted(img)
        warped = warp(undistort, mtx, dist)
        
        # draw src and dst points into original image and warped image respectively
        cv2.line(img, (src[0][0], src[0][1]), (src[1][0],  src[1][1]), [128,0,0], 5)
        cv2.line(img, (src[1][0], src[1][1]), (src[2][0],  src[2][1]), [128,0,0], 5)
        cv2.line(img, (src[2][0], src[2][1]), (src[3][0],  src[3][1]), [128,0,0], 5)
        cv2.line(img, (src[3][0], src[3][1]), (src[0][0],  src[0][1]), [128,0,0], 5)
        
        cv2.line(warped, (dst[0][0], dst[0][1]), (dst[1][0],  dst[1][1]), [0,0,128], 5)
        cv2.line(warped, (dst[1][0], dst[1][1]), (dst[2][0],  dst[2][1]), [0,0,128], 5)
        cv2.line(warped, (dst[2][0], dst[2][1]), (dst[3][0],  dst[3][1]), [0,0,128], 5)
        cv2.line(warped, (dst[3][0], dst[3][1]), (dst[0][0],  dst[0][1]), [0,0,128], 5)
        
        #f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
        #f.tight_layout()
        #ax1.imshow(img)
        #ax1.set_title('Original Image', fontsize=50)
        #ax2.imshow(warped)
        #ax2.set_title('Undistorted and Warped Image', fontsize=50)
        #plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
        
        r,g,b = cv2.split(img)  #since image gets recombined as BGR images, use cv2.split to put into RGB format
        img = cv2.merge((b,g,r))
        
        
        cv2.imwrite("output_images/output_orig_src_"+file,img)    #write output image
        cv2.imwrite("output_images/output_warped_dst_"+file,warped) #write output image

## Identify lane pixels in the rectified image and fit with a polynomial

Extend pipeline with histogram function and polynomial fit functions

In [8]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

# Define conversions in x and y from pixels space to meters
ym_per_pix = 30/720 # meters per pixel in y dimension (based on dst the full height of the warped image)
xm_per_pix = 3.7/640 # meters per pixel in x dimension (based on dst there are 600 pixels for the lane width)

def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    
    # Convert fit points to integer for drawing into image
    left_fitx_int = left_fitx.astype(int)
    right_fitx_int = right_fitx.astype(int)
    ploty_int = ploty.astype(int)

    # Plots the left and right polynomials on the lane lines
    out_img[ploty_int, left_fitx_int] = [0, 255, 0]
    out_img[ploty_int, right_fitx_int] = [0, 255, 0]

    return left_fit, right_fit, ploty, left_fitx, right_fitx, out_img

In [9]:
# Make a list of the distorted test images
files = os.listdir("test_images/")

# Step through the image list and write back each test image as an undistorted and binary image
for file in files:
    if file[0:6] != "output_": #execute only on files that were not already created and have the prefix "output"
        img = mpimg.imread("test_images/"+file)
        undistort = undistorted(img)
        binary = binary_img(undistort)
        color_binary = np.dstack((binary, binary, binary)) * 255 # stack each channel for RGB output
        warped = warp(color_binary, mtx, dist)
        warped_binary = warped[:,:,0]
        left_fit, right_fit, ploty, left_fitx, right_fitx, out_polynomial = fit_polynomial(warped_binary)
        cv2.imwrite("output_images/output_binary_polynomial_"+file,out_polynomial)    #write output image

## Calculate the radius of curvature and vehicle position with respect to the center

In [10]:
def measure_curvature_real(binary_warped):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = 719 # y-value at bottom of the image
    
    ##### the calculation of R_curve (radius of curvature) #####
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad

def measure_vehicle_distance2center(left_fitx, right_fitx, ploty):
    '''
    Calculates the vehicle distance to the center of the lane
    '''
    
    # Define y-value where we want to calculate the vehicle distance to the center of the road
    # That is the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty).astype(int)
    
    offset = ((left_fitx[y_eval]+right_fitx[y_eval])/2 - 640)*xm_per_pix

    return offset
    

In [11]:
# Make a list of the distorted test images
files = os.listdir("test_images/")

# Step through the image list and write back each test image as an undistorted and binary image
for file in files:
    if file[0:6] != "output_": #execute only on files that were not already created and have the prefix "output"
        img = mpimg.imread("test_images/"+file)
        undistort = undistorted(img)
        binary = binary_img(undistort)
        color_binary = np.dstack((binary, binary, binary)) * 255 # stack each channel for RGB output
        warped = warp(color_binary, mtx, dist)
        warped_binary = warped[:,:,0]
        left_fit, right_fit, ploty, left_fitx, right_fitx, out_polynomial = fit_polynomial(warped_binary)
        # Calculate the radius of curvature in meters for both lane lines
        left_curverad, right_curverad = measure_curvature_real(warped_binary)
        avg_curverad = (left_curverad+right_curverad)//2
        #print(left_curverad, 'm     ', right_curverad, 'm\n')
        offset = measure_vehicle_distance2center(left_fitx, right_fitx, ploty)
        #print(offset, 'm\n\n')
        cv2.putText(out_polynomial,"Avg_Curvature: {} m".format(avg_curverad), (500,80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(out_polynomial,"Offset: {:.2f} m".format(offset), (500,120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        cv2.imwrite("output_images/output_binary_polynomial_with_cr_"+file,out_polynomial)    #write output image

## Warp the detected lane boundaries back onto the original image

In [12]:
# Make a list of the distorted test images
files = os.listdir("test_images/")

# Step through the image list and write back each test image as an undistorted and binary image
for file in files:
    if file[0:6] != "output_": #execute only on files that were not already created and have the prefix "output"
        img = mpimg.imread("test_images/"+file)
        undistort = undistorted(img)
        binary = binary_img(undistort)
        color_binary = np.dstack((binary, binary, binary)) * 255 # stack each channel for RGB output
        warped = warp(color_binary, mtx, dist)
        warped_binary = warped[:,:,0]
        left_fit, right_fit, ploty, left_fitx, right_fitx, out_polynomial = fit_polynomial(warped_binary)
        # Create an image to draw the lines on
        warp_zero = np.zeros_like(warped_binary).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
        # Combine the result with the original image
        result = cv2.addWeighted(undistort, 1, newwarp, 0.3, 0)
        #plt.imshow(result)
        # Calculate the radius of curvature in meters for both lane lines
        left_curverad, right_curverad = measure_curvature_real(warped_binary)
        avg_curverad = (left_curverad+right_curverad)//2
        #print(left_curverad, 'm     ', right_curverad, 'm\n')
        offset = measure_vehicle_distance2center(left_fitx, right_fitx, ploty)
        #print(offset, 'm\n\n')
        cv2.putText(result,"Avg_Curvature: {} m".format(avg_curverad), (500,80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(result,"Offset: {:.2f} m".format(offset), (500,120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        cv2.imwrite("output_images/output_result_"+file,result)    #write output image

This finishes the pipeline for the still test images.

## Build the pipeline for video

Camera calibration and calculating the perspective transform matrices based on hardcoded src and dst points

In [13]:
# prepare object points for a 9x6 chess board with z=0, like (0,0,0), (1,0,0), ...
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

# create empty object points and image points arrays
objpoints = []
imgpoints = [] 

# Make a list of calibration images
files = os.listdir("camera_cal/")

# Step through the image list and search for corners
for file in files:
    if file[0:11] == "calibration": #execute only on files with the prefix "calibration"
        img = mpimg.imread("camera_cal/"+file)
        img_size = (img.shape[1], img.shape[0])
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # convert to grayscale from RGB as were read with mpimg

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9, 6), None)
    
        # If found, draw corners, compute calibration and save images to folder
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
        
# Compute the camera calibration and distortion coefficients
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)

# Undistort a test image with the parameters from cv2.calibrateCamera function
img = mpimg.imread("camera_cal/calibration2.jpg")
img = cv2.undistort(img, mtx, dist, None, mtx)
r,g,b = cv2.split(img)    #since image gets recombined as BGR images, use cv2.split to put into RGB format
img = cv2.merge((b,g,r))
cv2.imwrite("camera_cal/undistort_calibration2.jpg", img);

# Use hardcoded course and destination points
src = np.float32(
    [[img_size[0] / 2 - 55, img_size[1] / 2 + 100],
    [img_size[0] / 6 - 10, img_size[1]],
    [img_size[0] * 5 / 6 + 60, img_size[1]],
    [img_size[0] / 2 + 55, img_size[1] / 2 + 100]])
dst = np.float32(
    [[img_size[0] / 4, 0],
    [img_size[0] / 4, img_size[1]],
    [img_size[0] * 3 / 4, img_size[1]],
    [img_size[0] * 3 / 4, 0]])

# Given src and dst points, calculate the perspective transform matrices
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)

Pipeline with individual functions for later processing

In [14]:
def undistorted(img):
    # Use calibration parameters calculated before in cv2.undistort()
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

def binary_img(img, s_thresh=(254, 255), b_thresh=(254, 255), sx_thresh=(20, 80)):
    # Use contrast correction to fight excessive darkness or brightness
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img[:,:,0] = cv2.equalizeHist(img[:,:,0])
    img = cv2.cvtColor(img, cv2.COLOR_YUV2RGB) 
    
    # Convert to HLS color space and separate the S channels
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    # Convert to LAB color space and separate the B channel 
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    b_channel = lab[:,:,2]
    
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold S-color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # Threshold B-color channel
    b_binary = np.zeros_like(b_channel)
    b_binary[(b_channel >= b_thresh[0]) & (b_channel <= b_thresh[1])] = 1
    
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (b_binary == 1) | (sxbinary == 1)] = 1
    
    return combined_binary

# Define a function that takes an image, number of x and y points, 
# camera matrix and distortion coefficients
def warp(img, mtx, dist):
    # Use the OpenCV undistort() function to remove distortion
    undist = cv2.undistort(img, mtx, dist, None, mtx)
        
    img_size = (img.shape[1], img.shape[0])
        
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(undist, M, img_size)
    
    # Return the resulting image and matrix
    return warped

def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    
    # Convert fit points to integer for drawing into image
    #left_fitx_int = left_fitx.astype(int)
    #right_fitx_int = right_fitx.astype(int)
    #ploty_int = ploty.astype(int)

    # Plots the left and right polynomials on the lane lines
    #out_img[ploty_int, left_fitx_int] = [0, 255, 0]
    #out_img[ploty_int, right_fitx_int] = [0, 255, 0]

    return left_fit, right_fit, ploty, left_fitx, right_fitx, out_img

def measure_curvature_real(binary_warped):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = 719 # y-value at bottom of the image
    
    ##### the calculation of R_curve (radius of curvature) #####
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad

def measure_vehicle_distance2center(left_fitx, right_fitx, ploty):
    '''
    Calculates the vehicle distance to the center of the lane
    '''
    
    # Define y-value where we want to calculate the vehicle distance to the center of the road
    # That is the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty).astype(int)
    
    offset = ((left_fitx[y_eval]+right_fitx[y_eval])/2 - 640)*xm_per_pix

    return offset

Importing everything needed to edit/save/watch video clips

In [15]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

The pipeline function to process each frame

In [16]:
def process_image(img):
    # The output you return should be a the undistorted image with the polygon drawn in between the detected lane lines
    
    undistort = undistorted(img)
    binary = binary_img(undistort)
    color_binary = np.dstack((binary, binary, binary)) * 255 # stack each channel for RGB output
    kernel = np.ones((2,2),np.uint8)
    color_binary = cv2.morphologyEx(color_binary, cv2.MORPH_OPEN, kernel)
    warped = warp(color_binary, mtx, dist)
    warped_binary = warped[:,:,0]
    left_fit, right_fit, ploty, left_fitx, right_fitx, out_polynomial = fit_polynomial(warped_binary)
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped_binary).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undistort, 1, newwarp, 0.3, 0)
    #plt.imshow(result)
    # Calculate the radius of curvature in meters for both lane lines
    left_curverad, right_curverad = measure_curvature_real(warped_binary)
    avg_curverad = (left_curverad+right_curverad)//2
    #print(left_curverad, 'm     ', right_curverad, 'm\n')
    offset = measure_vehicle_distance2center(left_fitx, right_fitx, ploty)
    #print(offset, 'm\n\n')
    cv2.putText(result,"Avg_Curvature: {} m".format(avg_curverad), (500,80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    cv2.putText(result,"Offset: {:.2f} m".format(offset), (500,120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    #cv2.imwrite("output_images/output_result_"+file,result)    #write output image
    
    return result

Using the pipeline on the `project video`

In [17]:
project_goal_video_output = "output_test_videos/output_project_video.mp4"
#clip1 = VideoFileClip("test_videos/project_video.mp4").subclip(0,1)
clip1 = VideoFileClip("test_videos/project_video.mp4")
video_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time video_clip.write_videofile(project_goal_video_output, audio=False)

[MoviePy] >>>> Building video output_test_videos/output_project_video.mp4
[MoviePy] Writing video output_test_videos/output_project_video.mp4


100%|█████████▉| 1260/1261 [04:34<00:00,  4.48it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_test_videos/output_project_video.mp4 

Wall time: 4min 35s


In [18]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_goal_video_output))

Using the pipeline on the `challenge video`

In [19]:
project_challenge_video_output = "output_test_videos/output_challenge_video.mp4"
#clip1 = VideoFileClip("test_videos/challenge_video.mp4").subclip(0,1)
clip1 = VideoFileClip("test_videos/challenge_video.mp4")
video_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time video_clip.write_videofile(project_challenge_video_output, audio=False)

[MoviePy] >>>> Building video output_test_videos/output_challenge_video.mp4
[MoviePy] Writing video output_test_videos/output_challenge_video.mp4


100%|██████████| 485/485 [01:42<00:00,  4.82it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_test_videos/output_challenge_video.mp4 

Wall time: 1min 43s


In [20]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_challenge_video_output))

Using the pipeline on the `harder challenge video`

In [21]:
project_harder_challenge_video_output = "output_test_videos/output_harder_challenge_video.mp4"
#clip1 = VideoFileClip("test_videos/harder_challenge_video.mp4").subclip(0,1)
clip1 = VideoFileClip("test_videos/harder_challenge_video.mp4")
video_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time video_clip.write_videofile(project_harder_challenge_video_output, audio=False)

[MoviePy] >>>> Building video output_test_videos/output_harder_challenge_video.mp4
[MoviePy] Writing video output_test_videos/output_harder_challenge_video.mp4


100%|█████████▉| 1199/1200 [05:45<00:00,  4.32it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_test_videos/output_harder_challenge_video.mp4 

Wall time: 5min 47s


In [22]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_harder_challenge_video_output))

## Improving the steadiness of the detection by filtering

Pipeline with individual functions for processing of the frames

In [17]:
def undistorted(img):
    # Use calibration parameters calculated before in cv2.undistort()
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

def binary_img(img, s_thresh=(254, 255), b_thresh=(254, 255), sx_thresh=(20, 80)):
    # Use contrast correction to fight excessive darkness or brightness
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img[:,:,0] = cv2.equalizeHist(img[:,:,0])
    img = cv2.cvtColor(img, cv2.COLOR_YUV2RGB) 
    
    # Convert to HLS color space and separate the S channels
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    # Convert to LAB color space and separate the B channel 
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    b_channel = lab[:,:,2]
    
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold S-color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # Threshold B-color channel
    b_binary = np.zeros_like(b_channel)
    b_binary[(b_channel >= b_thresh[0]) & (b_channel <= b_thresh[1])] = 1
    
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (b_binary == 1) | (sxbinary == 1)] = 1
    
    return combined_binary

# Define a function that takes an image, number of x and y points, 
# camera matrix and distortion coefficients
def warp(img, mtx, dist):
    # Use the OpenCV undistort() function to remove distortion
    undist = cv2.undistort(img, mtx, dist, None, mtx)
        
    img_size = (img.shape[1], img.shape[0])
        
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(undist, M, img_size)
    
    # Return the resulting image and matrix
    return warped

def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    
    # Convert fit points to integer for drawing into image
    #left_fitx_int = left_fitx.astype(int)
    #right_fitx_int = right_fitx.astype(int)
    #ploty_int = ploty.astype(int)

    # Plots the left and right polynomials on the lane lines
    #out_img[ploty_int, left_fitx_int] = [0, 255, 0]
    #out_img[ploty_int, right_fitx_int] = [0, 255, 0]

    return left_fit, right_fit, ploty, left_fitx, right_fitx, out_img

def measure_curvature_real(binary_warped):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = 719 # y-value at bottom of the image
    
    ##### the calculation of R_curve (radius of curvature) #####
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad

def measure_vehicle_distance2center(left_fitx, right_fitx, ploty):
    '''
    Calculates the vehicle distance to the center of the lane
    '''
    
    # Define y-value where we want to calculate the vehicle distance to the center of the road
    # That is the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty).astype(int)
    
    offset = ((left_fitx[y_eval]+right_fitx[y_eval])/2 - 640)*xm_per_pix

    return offset

In [18]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        #self.detected = False  
        # x values of the last n fits of the line
        #self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        #self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = np.array([None]) 
        #polynomial coefficients for the most recent fit
        self.current_fit = np.array(None)  
        #radius of curvature of the line in some units
        self.radius_of_curvature = 0 
        #distance in meters of vehicle center from the line
        self.line_base_pos = 0 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        #self.allx = None  
        #y values for detected line pixels
        #self.ally = None
        self.framecount = 0

In [19]:
def process_image_filter(img):
    # The output you return should be a the undistorted image with the polygon drawn in between the detected lane lines
    
    undistort = undistorted(img)
    binary = binary_img(undistort)
    color_binary = np.dstack((binary, binary, binary)) * 255 # stack each channel for RGB output
    kernel = np.ones((2,2),np.uint8)
    color_binary = cv2.morphologyEx(color_binary, cv2.MORPH_OPEN, kernel)
    warped = warp(color_binary, mtx, dist)
    warped_binary = warped[:,:,0]
    left_fit, right_fit, ploty, left_fitx, right_fitx, out_polynomial = fit_polynomial(warped_binary)
    # Calculate the radius of curvature in meters for both lane lines
    left_curverad, right_curverad = measure_curvature_real(warped_binary)    
    # Calculate the vehicle offset
    offset = measure_vehicle_distance2center(left_fitx, right_fitx, ploty)
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped_binary).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    filter_coeff = 0.95
    
    if left_line.framecount > 0:
        left_line.current_fit = left_fit
        left_line.best_fit = filter_coeff*left_line.best_fit + (1-filter_coeff)*left_line.current_fit
        left_line.radius_of_curvature = filter_coeff*left_line.radius_of_curvature + (1-filter_coeff)*left_curverad
        left_line.line_base_pos = filter_coeff*left_line.line_base_pos + (1-filter_coeff)*offset
    else:
        left_line.best_fit = left_fit
        left_line.radius_of_curvature = left_curverad
        left_line.offset = offset
        left_line.framecount += 1
        
    if right_line.framecount > 0:
        right_line.current_fit = right_fit
        right_line.best_fit = filter_coeff*right_line.best_fit + (1-filter_coeff)*right_line.current_fit
        right_line.radius_of_curvature = filter_coeff*right_line.radius_of_curvature + (1-filter_coeff)*right_curverad
        right_line.line_base_pos = filter_coeff*right_line.line_base_pos + (1-filter_coeff)*offset
    else:
        right_line.best_fit = right_fit
        right_line.radius_of_curvature = right_curverad
        right_line.offset = offset
        right_line.framecount += 1
        
    avg_curverad = (left_line.radius_of_curvature + right_line.radius_of_curvature)/2
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, color_warp.shape[0]-1, color_warp.shape[0] )
    try:
        left_fitx = left_line.best_fit[0]*ploty**2 + left_line.best_fit[1]*ploty + left_line.best_fit[2]
        right_fitx = right_line.best_fit[0]*ploty**2 + right_line.best_fit[1]*ploty + right_line.best_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undistort, 1, newwarp, 0.3, 0)

    
    #print(offset, 'm\n\n')
    cv2.putText(result,"Avg_Curvature: {:.0f} m".format(avg_curverad), (500,80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    cv2.putText(result,"Offset: {:.2f} m".format(offset), (500,120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    #cv2.imwrite("output_images/output_result_"+file,result)    #write output image
    
    return result

In [20]:
left_line = Line()
right_line = Line()

project_goal_filter_video_output = "output_test_videos/output_project_filter_video.mp4"
#clip1 = VideoFileClip("test_videos/project_video.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/project_video.mp4")
video_clip = clip1.fl_image(process_image_filter) #NOTE: this function expects color images!!
%time video_clip.write_videofile(project_goal_filter_video_output, audio=False)

[MoviePy] >>>> Building video output_test_videos/output_project_filter_video.mp4
[MoviePy] Writing video output_test_videos/output_project_filter_video.mp4


100%|█████████▉| 1260/1261 [05:08<00:00,  4.71it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_test_videos/output_project_filter_video.mp4 

Wall time: 5min 9s


In [21]:
left_line = Line()
right_line = Line()

project_challenge_filter_video_output = "output_test_videos/output_challenge_filter_video.mp4"
#clip1 = VideoFileClip("test_videos/challenge_video.mp4").subclip(0,1)
clip1 = VideoFileClip("test_videos/challenge_video.mp4")
video_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time video_clip.write_videofile(project_challenge_filter_video_output, audio=False)

[MoviePy] >>>> Building video output_test_videos/output_challenge_filter_video.mp4
[MoviePy] Writing video output_test_videos/output_challenge_filter_video.mp4


100%|██████████| 485/485 [01:36<00:00,  5.19it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_test_videos/output_challenge_filter_video.mp4 

Wall time: 1min 37s


In [22]:
left_line = Line()
right_line = Line()

project_harder_challenge_filter_video_output = "output_test_videos/output_harder_challenge_filter_video.mp4"
#clip1 = VideoFileClip("test_videos/harder_challenge_video.mp4").subclip(0,1)
clip1 = VideoFileClip("test_videos/harder_challenge_video.mp4")
video_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time video_clip.write_videofile(project_harder_challenge_filter_video_output, audio=False)

[MoviePy] >>>> Building video output_test_videos/output_harder_challenge_filter_video.mp4
[MoviePy] Writing video output_test_videos/output_harder_challenge_filter_video.mp4


100%|█████████▉| 1199/1200 [05:10<00:00,  4.01it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_test_videos/output_harder_challenge_filter_video.mp4 

Wall time: 5min 12s
